In [13]:
import requests 
import re, time
from bs4 import BeautifulSoup 
import pandas as pd
import pymongo 
from nltk.corpus import stopwords
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import indeed_query

In [14]:
executable_path = {'executable_path': ChromeDriverManager().install()}

[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\Tim\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [3]:
import nltk
 #nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  

In [22]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.indeed
collection = db.posts

base_url='https://www.indeed.com/'
qloc='Washington+DC'
query=indeed_query
query = query.replace(" ", "%20")
query

'C%23+net+-clearance'

In [25]:

for i in range(0,100,10):
    url = f"{base_url}jobs?q={query}&l={qloc}&jt=fulltime&start={i}"  
    soup = BeautifulSoup(requests.get(url).text, "html.parser")   
    els = soup.select('.jobsearch-SerpJobCard') 
    print(url)
    time.sleep(1.5) 
    
    if len(els) < 1:
        continue
    for e in els: 
        if len(e) < 1:
            break
        dic = {} 
        dic['query']=query
        dic['qloc']=qloc
        dic['title']=e.select('.title')[0].text.replace('\n','')
        dic['company']=e.select('.company')[0].text.replace('\n','')
        dic['location']=e.select('.location')[0].text.replace('\n','')
        dic['summary']=e.select('.summary')[0].text.replace('\n','')
        dic['entry']='-'.join([c for c in re.findall('\w*', f"{dic['title']}{dic['company']}") if c != '' and c not in stop_words])
        dic['date']=e.select('.date')[0].text.replace('\n','')
        if len(e.select('.salaryText')) > 0: 
            salaryText=e.select('.salaryText')[0].text.replace('\n','')
            dic['salary'] = re.findall('\d{2,3}\,\d{1,3}',salaryText) 
            if len(dic['salary']) > 1:
                dic['salarymin'] =  dic['salary'][0]
                dic['salarymax'] =  dic['salary'][1] 
        collection.delete_one(  {'entry': dic['entry']}  )
        collection.insert_one(dic) 
 


https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=0
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=10
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=20
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=30
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=40
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=50
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=60
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=70
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=80
https://www.indeed.com/jobs?q=C%23+net+-clearance&l=Washington+DC&jt=fulltime&start=90


In [26]:
df = pd.DataFrame(collection.find({}))
try:
    df['salarymin'] = df['salarymin'].apply(lambda x: str(x).replace(',', '')).astype('float') 
except:
    pass
df['url'] = 'https://www.google.com/search?q=' + df['company'] + '+Careers'
df['url'] = df['url'].apply(lambda x: str(x).replace(' ', '%20'))
try:
    df['date'] = df['date'].apply(lambda x: re.sub('days ago|day ago|\+|Today|\s', '', x)).replace('','0').astype('float') 
except:
    pass

df.sort_values(by='date', ascending=True, inplace=True) 
#df = df.loc[df.title.str.contains('Developer|Engineer|Software', regex=True)]
for i, r in df.iterrows():
    print(f"{r['title']} {r['company']} {r['url']}" )
df

Software Developer (.Net and Azure)new Exostar LLC https://www.google.com/search?q=Exostar%20LLC+Careers
.NET Software Developernew Copper River Infrastructure Services https://www.google.com/search?q=Copper%20River%20Infrastructure%20Services+Careers
.Net Developernew FAAZ Consulting https://www.google.com/search?q=FAAZ%20Consulting+Careers
Software Developer (.Net and Azure)new Exostar https://www.google.com/search?q=Exostar+Careers
Sharepoint Developer ICF https://www.google.com/search?q=ICF+Careers
Senior Applications Developer NucoreVision (NVI) https://www.google.com/search?q=NucoreVision%20(NVI)+Careers
Developer - Python / C# / Azure experience JM Talent Inc. https://www.google.com/search?q=JM%20Talent%20Inc.+Careers
Software Engineer Casepoint LLC https://www.google.com/search?q=Casepoint%20LLC+Careers
Software Development Advisor RadiumSoft, Inc https://www.google.com/search?q=RadiumSoft,%20Inc+Careers
Senior .Net Developer Motion Recruitment https://www.google.com/search?q=M

,_id,query,qloc,title,company,location,summary,entry,date,salary,salarymin,salarymax,url
156,60a6e039c07a2658049f232f,C%23+net+-clearance,Washington+DC,Software Developer (.Net and Azure)new,Exostar LLC,"Herndon, VA 20171",Position Title: Software Developer (.Net and A...,Software-Developer-Net-Azure-newExostar-LLC,1 day ago,NaN,NaN,NaN,https://www.google.com/search?q=Exostar%20LLC+...
82,60a6e02dc07a2658049f22d7,C%23+net+-clearance,Washington+DC,.NET Software Developernew,Copper River Infrastructure Services,"Fort Belvoir, VA","Your health is most important, so you’ll be “c...",NET-Software-DevelopernewCopper-River-Infrastr...,1 day ago,NaN,NaN,NaN,https://www.google.com/search?q=Copper%20River...
70,60a6e02bc07a2658049f22c9,C%23+net+-clearance,Washington+DC,.Net Developernew,FAAZ Consulting,"Arlington, VA","Nice to haves*: Latest versions of .NET core, ...",Net-DevelopernewFAAZ-Consulting,1 day ago,"[115,000]",NaN,NaN,https://www.google.com/search?q=FAAZ%20Consult...
37,60a6e01cc07a2658049f229c,C%23+net+-clearance,Washington+DC,Software Developer (.Net and Azure)new,Exostar,"Herndon, VA 20171",Position Title: * Software Developer (.Net and...,Software-Developer-Net-Azure-newExostar,1 day ago,NaN,NaN,NaN,https://www.google.com/search?q=Exostar+Careers
116,60a6e033c07a2658049f2305,C%23+net+-clearance,Washington+DC,Sharepoint Developer,ICF,"Bethesda, MD","As a Sharepoint Developer, you will be respons...",Sharepoint-DeveloperICF,10 days ago,NaN,NaN,NaN,https://www.google.com/search?q=ICF+Careers
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52,60a6e027c07a2658049f22b1,C%23+net+-clearance,Washington+DC,100% Remote Senior C# Developernew,CyberCoders,"Washington, DC 20036 (Downtown area)",Apply design patterns and architecture to our ...,100-Remote-Senior-C-DevelopernewCyberCoders,Just posted,"[100,000, 160,000]",100000.0,"160,000",https://www.google.com/search?q=CyberCoders+Ca...
43,60a6e01cc07a2658049f22a4,C%23+net+-clearance,Washington+DC,Angular Full Stack Developernew,CACI,"Ashburn, VA 20147",Serve as an Agile Scrum team member providing ...,Angular-Full-Stack-DevelopernewCACI,Today,NaN,NaN,NaN,https://www.google.com/search?q=CACI+Careers
104,60a6e031c07a2658049f22f9,C%23+net+-clearance,Washington+DC,Senior Full Stack Engineer (Front-end Focus) -...,FrontStream,"Reston, VA 20191 (South Lakes Dr - Soapstone D...",You will have the opportunity to work on moder...,Senior-Full-Stack-Engineer-Front-end-Focus-Res...,Today,NaN,NaN,NaN,https://www.google.com/search?q=FrontStream+Ca...
2,60a6de88c07a2658049f2230,C%23+net+-clearance,Washington+DC,Sr. C# .NET Developernew,Consumers' Checkbook/Center for the Study of S...,"Washington, DC 20006 (Downtown area)","Experience with HTML5, JavaScript/TypeScript, ...",Sr-C-NET-DevelopernewConsumers-Checkbook-Cente...,Today,"[98,000, 140,000]",98000.0,"140,000",https://www.google.com/search?q=Consumers'%20C...


In [27]:
conn = 'mongodb://localhost:27017' 
cursor = collection.find( { }, {'_id': False})
df = pd.DataFrame(cursor)
df.salarymin=df.salarymin.fillna('') 
df.loc[df.salarymin != '']

,query,qloc,title,company,location,summary,entry,date,salary,salarymin,salarymax
2,C%23+net+-clearance,Washington+DC,Sr. C# .NET Developernew,Consumers' Checkbook/Center for the Study of S...,"Washington, DC 20006 (Downtown area)","Experience with HTML5, JavaScript/TypeScript, ...",Sr-C-NET-DevelopernewConsumers-Checkbook-Cente...,Today,"[98,000, 140,000]","98,000","140,000"
5,C%23+net+-clearance,Washington+DC,Staff Backend Engineer,Pie Insurance,"Washington, DC","As a Staff Engineer at Pie Insurance, you’ll b...",Staff-Backend-EngineerPie-Insurance,30+ days ago,"[145,000, 203,000]","145,000","203,000"
7,C%23+net+-clearance,Washington+DC,US-based C# .Net Developer,Purefy,"Washington, DC 20009 (Adams Morgan area)",Union and other leading lenders to deliver a f...,US-based-C-Net-DeveloperPurefy,30+ days ago,"[85,000, 110,000]","85,000","110,000"
22,C%23+net+-clearance,Washington+DC,Sr. Applications Developer,"Data Base Connection, Inc","Washington, DC 20006 (Foggy Bottom area)",NET C# developer with more than seven (7) year...,Sr-Applications-DeveloperData-Base-Connection-Inc,Active 2 days ago,"[120,000, 130,000]","120,000","130,000"
41,C%23+net+-clearance,Washington+DC,Sr. Web Developer,"Virid, Inc.","Reston, VA 20191 (Hattontown area)",The developer will be part of cross-functional...,Sr-Web-DeveloperVirid-Inc,30+ days ago,"[78,000, 140,000]","78,000","140,000"
48,C%23+net+-clearance,Washington+DC,Developer - Python / C# / Azure experience,JM Talent Inc.,"Washington, DC","As part of this initiative, teams will use mac...",Developer-Python-C-Azure-experienceJM-Talent-Inc,10 days ago,"[93,000, 125,000]","93,000","125,000"
49,C%23+net+-clearance,Washington+DC,Senior .Net Developer,Motion Recruitment,"Arlington, VA 22201",Their system is used for tracking their sales ...,Senior-Net-DeveloperMotion-Recruitment,13 days ago,"[100,000, 125,000]","100,000","125,000"
51,C%23+net+-clearance,Washington+DC,SR. Software Engineer - C#/.NET (JO-2104-11602...,Ledgent Technology,"Washington, DC 20007 (Georgetown area)",Assist in planning and coordination of the sof...,SR-Software-Engineer-C-NET-JO-2104-116028-newL...,2 days ago,"[100,000, 140,000]","100,000","140,000"
52,C%23+net+-clearance,Washington+DC,100% Remote Senior C# Developernew,CyberCoders,"Washington, DC 20036 (Downtown area)",Apply design patterns and architecture to our ...,100-Remote-Senior-C-DevelopernewCyberCoders,Just posted,"[100,000, 160,000]","100,000","160,000"
72,C%23+net+-clearance,Washington+DC,Senior Backend Engineer,Pie Insurance,"Washington, DC","As a Senior Engineer at Pie Insurance, you’ll ...",Senior-Backend-EngineerPie-Insurance,30+ days ago,"[124,000, 180,000]","124,000","180,000"


In [28]:
 
collection.delete_many(
     { 'entry': {'$regex': '^.*$', '$options': 'i'}}
)
 

In [ ]:
browser = Browser('chrome', **executable_path, headless=False)

base_url='https://www.indeed.com/'
qloc='Washington+DC'
query='junior+data+python'
for i in range(0,20,10): 
    browser.visit(f"{base_url}jobs?q={query}&l={qloc}&jt=fulltime&start={i}")
    soup = BeautifulSoup( browser.html , 'html.parser') 
    for ele in soup.select('.jobsearch-SerpJobCard a.jobtitle'): 
        browser.click_link_by_id(f"{ele['id']}")

        with browser.get_iframe('vjs-container-iframe') as iframe: 
            soup = BeautifulSoup( iframe.html , 'html.parser')
            print(soup.select('#jobDescriptionText') == None)  